# Convolutional Neural Network for Malaria Cell Images 

In [1]:
# import dependencies
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Dropout,Dense,Flatten,BatchNormalization,Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.callbacks import ModelCheckpoint
%matplotlib inline
import glob
import cv2

Using TensorFlow backend.


In [2]:
# read in malaria cell images
infected = glob.glob('../Datasets/cell_images/Parasitized/*.png')
uninfected = glob.glob('../Datasets/cell_images/Uninfected/*.png')

In [ ]:
# 5,000 infected images and 5,000 uninfected images
print(f"{len(infected)} infected cell images")
print(f"{len(uninfected)} uninfected cell images")
print(f"Shape of first infected cell image: {cv2.imread(infected[0]).shape}")
print(f"Shape of first uninfected cell image: {cv2.imread(uninfected[0]).shape}")

In [ ]:
# exhibit 4 sample images of infected cells
plt.figure(figsize=(12,5))
for i in range(1,5):
    plt.subplot(1,4,i)
    value = np.random.randint(100)
    image = cv2.imread(infected[value])
    plt.imshow(image)
    plt.title('Infected Image')
    plt.xticks([])
    plt.yticks([])

In [ ]:
# exhibit 4 sample images of uninfected cells
plt.figure(figsize=(12,5))
for i in range(1,5):
    plt.subplot(1,4,i)
    value = np.random.randint(100)
    image = cv2.imread(uninfected[value])
    plt.imshow(image)
    plt.title('Uninfected Image')
    plt.xticks([])
    plt.yticks([])

In [12]:
# The original images consist in RGB coefficients in the 0-255, but such values would be too high for our model to process
# (given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255
augmentor = ImageDataGenerator(rescale=1./255, validation_split=0.3)

In [13]:
# create the training and testing datasets
train_generator = augmentor.flow_from_directory('../Datasets/cell_images/', batch_size=32,
                                                target_size = (96,96), class_mode = 'binary', subset = 'training')
test_generator = augmentor.flow_from_directory('../Datasets/cell_images/', batch_size=32, target_size=(96,96),
                                               class_mode='binary', subset='validation')

Found 19292 images belonging to 2 classes.
Found 8266 images belonging to 2 classes.


In [ ]:
model1 = Sequential()
model1.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))

# batch normalization allows each layer of a network to learn by itself a little bit more independently of other layers
# reduces overfitting
model1.add(BatchNormalization())

# max pooling helps with overfitting by reducing sample's dimensionality and allowing for assumptions 
# to be made about features contained in the sub-regions binned
model1.add(MaxPooling2D(2,2))

# dropout keeps a certain proportion of nodes from being used each go around (help prevent overfitting)
model1.add(Dropout(0.4))

model1.add(Convolution2D(32,(3,3),activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(2,2))
model1.add(Dropout(0.4))
model1.add(Convolution2D(64,(3,3),activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(2,2))
model1.add(Dropout(0.4))

# flatten changes the dimensions of the data to make it an input layer for the artificial neural network
model1.add(Flatten())

# A dense layer is used to change the dimensions of the vector. 
# Mathematically speaking, it applies a rotation, scaling, translation transform to the vector.
model1.add(Dense(64,activation='relu'))
model1.add(Dropout(0.4))

# The Sigmoid function takes any range real number and returns the output value which falls in the range of 0 to 1.
model1.add(Dense(1,activation='sigmoid'))

# compile the model
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history_custom = model1.fit_generator(train_generator, steps_per_epoch=100,
                              epochs = 5,validation_data=test_generator, validation_steps=10)

In [ ]:
model1.save("Malaria_CNN_Trained1.h5")

In [5]:
# removed a convolutional layer with 32 filters to speed up the model training process
# training and validation accuracies appear to be unstable (used 80/20 training validation split)
model2 = Sequential()
model2.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(2,2))
model2.add(Dropout(0.4))

model2.add(Convolution2D(64,(3,3),activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(2,2))
model2.add(Dropout(0.4))

model2.add(Flatten())

model2.add(Dense(64,activation='relu'))
model2.add(Dropout(0.4))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom2 = model2.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
200/200 [==============================] - 328s 2s/step - loss: 1.0576 - acc: 0.5481 - val_loss: 0.6509 - val_acc: 0.6328
Epoch 2/10
200/200 [==============================] - 336s 2s/step - loss: 0.6570 - acc: 0.5955 - val_loss: 0.6163 - val_acc: 0.6906
Epoch 3/10
200/200 [==============================] - 324s 2s/step - loss: 0.6452 - acc: 0.6005 - val_loss: 0.6314 - val_acc: 0.6781
Epoch 4/10
200/200 [==============================] - 334s 2s/step - loss: 0.6232 - acc: 0.6134 - val_loss: 0.6284 - val_acc: 0.6844
Epoch 5/10
200/200 [==============================] - 330s 2s/step - loss: 0.6052 - acc: 0.6308 - val_loss: 0.6677 - val_acc: 0.6531
Epoch 6/10
200/200 [==============================] - 329s 2s/step - loss: 0.5166 - acc: 0.7063 - va

In [6]:
model2.save("Malaria_CNN_Trained2.h5")

In [7]:
# changed the last hidden dense layer to 32 filters instead of 64
# training and validation accuracies still seem unstable (used 80/20 training validation split)
model3 = Sequential()
model3.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(2,2))
model3.add(Dropout(0.4))

model3.add(Convolution2D(64,(3,3),activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling2D(2,2))
model3.add(Dropout(0.4))

model3.add(Flatten())

model3.add(Dense(32,activation='relu'))
model3.add(Dropout(0.4))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom3 = model3.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 329s 2s/step - loss: 0.7726 - acc: 0.5372 - val_loss: 0.6810 - val_acc: 0.6797
Epoch 2/10
200/200 [==============================] - 316s 2s/step - loss: 0.6765 - acc: 0.5819 - val_loss: 0.7245 - val_acc: 0.6250
Epoch 3/10
200/200 [==============================] - 330s 2s/step - loss: 0.6685 - acc: 0.6017 - val_loss: 0.6233 - val_acc: 0.7047
Epoch 4/10
200/200 [==============================] - 314s 2s/step - loss: 0.6714 - acc: 0.6031 - val_loss: 0.6345 - val_acc: 0.6687
Epoch 5/10
200/200 [==============================] - 309s 2s/step - loss: 0.6639 - acc: 0.6023 - val_loss: 0.6670 - val_acc: 0.6125
Epoch 6/10
200/200 [==============================] - 308s 2s/step - loss: 0.6614 - acc: 0.5998 - val_loss: 0.6327 - val_acc: 0.7266
Epoch 7/10
200/200 [==============================] - 313s 2s/step - loss: 0.6585 - acc: 0.6000 - val_loss: 0.6682 - val_acc: 0.5749
Epoch 8/10
200/200 [==============================] - 308s 2s/step - 

In [8]:
model3.save("Malaria_CNN_Trained3.h5")

In [9]:
# added another hidden convolutional layer with 64 filters
# training and validation accuracies appear more stable now (used 80/20 training validation split)
model4 = Sequential()
model4.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(2,2))
model4.add(Dropout(0.4))

model4.add(Convolution2D(64,(3,3),activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(2,2))
model4.add(Dropout(0.4))

model4.add(Convolution2D(64,(3,3),activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling2D(2,2))
model4.add(Dropout(0.4))

model4.add(Flatten())

model4.add(Dense(64,activation='relu'))
model4.add(Dropout(0.4))
model4.add(Dense(1,activation='sigmoid'))
model4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom4 = model4.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 353s 2s/step - loss: 0.7144 - acc: 0.6023 - val_loss: 0.6885 - val_acc: 0.6406
Epoch 2/10
200/200 [==============================] - 350s 2s/step - loss: 0.6214 - acc: 0.6572 - val_loss: 0.5453 - val_acc: 0.7219
Epoch 3/10
200/200 [==============================] - 349s 2s/step - loss: 0.4530 - acc: 0.7909 - val_loss: 0.3465 - val_acc: 0.8688
Epoch 4/10
200/200 [==============================] - 343s 2s/step - loss: 0.2560 - acc: 0.9133 - val_loss: 0.3722 - val_acc: 0.9047
Epoch 5/10
200/200 [==============================] - 344s 2s/step - loss: 0.1934 - acc: 0.9383 - val_loss: 0.2402 - val_acc: 0.9169
Epoch 6/10
200/200 [==============================] - 331s 2s/step - loss: 0.2049 - acc: 0.9369 - val_loss: 0.2281 - val_acc: 0.9344
Epoch 7/10
200/200 [==============================] - 327s 2s/step - loss: 0.1939 - acc: 0.9417 - val_loss: 0.4439 - val_acc: 0.7484
Epoch 8/10
200/200 [==============================] - 327s 2s/step - 

In [10]:
model4.save("Malaria_CNN_Trained4.h5")

In [14]:
# training and validation accuracies appear stabilized now (used 70/30 training validation split)

model5 = Sequential()
model5.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model5.add(BatchNormalization())
model5.add(MaxPooling2D(2,2))
model5.add(Dropout(0.4))

model5.add(Convolution2D(64,(3,3),activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling2D(2,2))
model5.add(Dropout(0.4))

model5.add(Convolution2D(64,(3,3),activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling2D(2,2))
model5.add(Dropout(0.4))

model5.add(Flatten())

model5.add(Dense(64,activation='relu'))
model5.add(Dropout(0.4))
model5.add(Dense(1,activation='sigmoid'))
model5.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom5 = model5.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 339s 2s/step - loss: 0.6721 - acc: 0.6359 - val_loss: 0.8683 - val_acc: 0.5734
Epoch 2/10
200/200 [==============================] - 333s 2s/step - loss: 0.5125 - acc: 0.7494 - val_loss: 0.4784 - val_acc: 0.8422
Epoch 3/10
200/200 [==============================] - 336s 2s/step - loss: 0.3223 - acc: 0.8780 - val_loss: 0.2370 - val_acc: 0.9203
Epoch 4/10
200/200 [==============================] - 333s 2s/step - loss: 0.2581 - acc: 0.9089 - val_loss: 0.1749 - val_acc: 0.9422
Epoch 5/10
200/200 [==============================] - 332s 2s/step - loss: 0.2000 - acc: 0.9437 - val_loss: 0.1894 - val_acc: 0.9516
Epoch 6/10
200/200 [==============================] - 336s 2s/step - loss: 0.2042 - acc: 0.9363 - val_loss: 0.1755 - val_acc: 0.9609
Epoch 7/10
200/200 [==============================] - 334s 2s/step - loss: 0.1950 - acc: 0.9403 - val_loss: 0.1352 - val_acc: 0.9547
Epoch 8/10
200/200 [==============================] - 330s 2s/step - 

In [15]:
model5.save("Malaria_CNN_Trained5.h5")

In [16]:
# changed the last hidden dense layer from 64 to 32 filters
# training and validation accuracies appear stabilized now (used 70/30 training validation split)

model6 = Sequential()
model6.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model6.add(BatchNormalization())
model6.add(MaxPooling2D(2,2))
model6.add(Dropout(0.4))

model6.add(Convolution2D(64,(3,3),activation='relu'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D(2,2))
model6.add(Dropout(0.4))

model6.add(Convolution2D(64,(3,3),activation='relu'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D(2,2))
model6.add(Dropout(0.4))

model6.add(Flatten())

model6.add(Dense(32,activation='relu'))
model6.add(Dropout(0.4))
model6.add(Dense(1,activation='sigmoid'))
model6.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom6 = model6.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 343s 2s/step - loss: 0.6863 - acc: 0.5944 - val_loss: 1.0100 - val_acc: 0.4547
Epoch 2/10
200/200 [==============================] - 339s 2s/step - loss: 0.6206 - acc: 0.6464 - val_loss: 4.3948 - val_acc: 0.5375
Epoch 3/10
200/200 [==============================] - 352s 2s/step - loss: 0.3983 - acc: 0.8243 - val_loss: 0.4677 - val_acc: 0.8689
Epoch 4/10
200/200 [==============================] - 345s 2s/step - loss: 0.2624 - acc: 0.9069 - val_loss: 0.4070 - val_acc: 0.8469
Epoch 5/10
200/200 [==============================] - 347s 2s/step - loss: 0.2204 - acc: 0.9277 - val_loss: 0.3124 - val_acc: 0.9375
Epoch 6/10
200/200 [==============================] - 342s 2s/step - loss: 0.2266 - acc: 0.9329 - val_loss: 0.1581 - val_acc: 0.9516
Epoch 7/10
200/200 [==============================] - 343s 2s/step - loss: 0.1909 - acc: 0.9430 - val_loss: 0.4121 - val_acc: 0.8344
Epoch 8/10
200/200 [==============================] - 345s 2s/step - 

In [17]:
model6.save("Malaria_CNN_Trained6.h5")

In [18]:
# changed the number of filters in the layers to 32, 64, 32, 64
# training and validation accuracies appear stabilized now (used 70/30 training validation split)

model7 = Sequential()
model7.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model7.add(BatchNormalization())
model7.add(MaxPooling2D(2,2))
model7.add(Dropout(0.4))

model7.add(Convolution2D(64,(3,3),activation='relu'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D(2,2))
model7.add(Dropout(0.4))

model7.add(Convolution2D(32,(3,3),activation='relu'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D(2,2))
model7.add(Dropout(0.4))

model7.add(Flatten())

model7.add(Dense(64,activation='relu'))
model7.add(Dropout(0.4))
model7.add(Dense(1,activation='sigmoid'))
model7.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom7 = model7.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 325s 2s/step - loss: 0.7038 - acc: 0.6146 - val_loss: 1.1786 - val_acc: 0.5141
Epoch 2/10
200/200 [==============================] - 324s 2s/step - loss: 0.5510 - acc: 0.7205 - val_loss: 0.8871 - val_acc: 0.7344
Epoch 3/10
200/200 [==============================] - 337s 2s/step - loss: 0.3431 - acc: 0.8592 - val_loss: 0.4831 - val_acc: 0.8641
Epoch 4/10
200/200 [==============================] - 325s 2s/step - loss: 0.2538 - acc: 0.9097 - val_loss: 0.3298 - val_acc: 0.8531
Epoch 5/10
200/200 [==============================] - 326s 2s/step - loss: 0.2175 - acc: 0.9295 - val_loss: 0.1560 - val_acc: 0.9515
Epoch 6/10
200/200 [==============================] - 330s 2s/step - loss: 0.2011 - acc: 0.9400 - val_loss: 0.3269 - val_acc: 0.8922
Epoch 7/10
200/200 [==============================] - 331s 2s/step - loss: 0.1906 - acc: 0.9444 - val_loss: 0.1731 - val_acc: 0.9422
Epoch 8/10
200/200 [==============================] - 340s 2s/step - 

In [19]:
model7.save("Malaria_CNN_Trained7.h5")

In [20]:
# added another hidden convolutional layer with 32 filters
# model accuracies did not increase by a significant amount (used 70/30 training validation split)
# should stick with model 7 
model8 = Sequential()
model8.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model8.add(BatchNormalization())
model8.add(MaxPooling2D(2,2))
model8.add(Dropout(0.4))

model8.add(Convolution2D(64,(3,3),activation='relu'))
model8.add(BatchNormalization())
model8.add(MaxPooling2D(2,2))
model8.add(Dropout(0.4))

model8.add(Convolution2D(32,(3,3),activation='relu'))
model8.add(BatchNormalization())
model8.add(MaxPooling2D(2,2))
model8.add(Dropout(0.4))

model8.add(Convolution2D(32,(3,3),activation='relu'))
model8.add(BatchNormalization())
model8.add(MaxPooling2D(2,2))
model8.add(Dropout(0.4))

model8.add(Flatten())

model8.add(Dense(64,activation='relu'))
model8.add(Dropout(0.4))
model8.add(Dense(1,activation='sigmoid'))
model8.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom8 = model8.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 360s 2s/step - loss: 0.7311 - acc: 0.6184 - val_loss: 1.2160 - val_acc: 0.5875
Epoch 2/10
200/200 [==============================] - 358s 2s/step - loss: 0.4514 - acc: 0.7834 - val_loss: 0.5345 - val_acc: 0.8609
Epoch 3/10
200/200 [==============================] - 342s 2s/step - loss: 0.2590 - acc: 0.9050 - val_loss: 0.2621 - val_acc: 0.9219
Epoch 4/10
200/200 [==============================] - 353s 2s/step - loss: 0.2235 - acc: 0.9286 - val_loss: 0.2124 - val_acc: 0.9125
Epoch 5/10
200/200 [==============================] - 365s 2s/step - loss: 0.1995 - acc: 0.9384 - val_loss: 0.1766 - val_acc: 0.9391
Epoch 6/10
200/200 [==============================] - 345s 2s/step - loss: 0.1928 - acc: 0.9431 - val_loss: 0.1314 - val_acc: 0.9484
Epoch 7/10
200/200 [==============================] - 350s 2s/step - loss: 0.1887 - acc: 0.9409 - val_loss: 0.3819 - val_acc: 0.7328
Epoch 8/10
200/200 [==============================] - 359s 2s/step - 

In [21]:
model8.save("Malaria_CNN_Trained8.h5")

In [22]:
# changed the number of filters to 32, 64, 32, 64, 32
# model accuracies did not increase by a significant amount (used 70/30 training validation split)
# should stick with model 7 

model9 = Sequential()
model9.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model9.add(BatchNormalization())
model9.add(MaxPooling2D(2,2))
model9.add(Dropout(0.4))

model9.add(Convolution2D(64,(3,3),activation='relu'))
model9.add(BatchNormalization())
model9.add(MaxPooling2D(2,2))
model9.add(Dropout(0.4))

model9.add(Convolution2D(32,(3,3),activation='relu'))
model9.add(BatchNormalization())
model9.add(MaxPooling2D(2,2))
model9.add(Dropout(0.4))

model9.add(Convolution2D(64,(3,3),activation='relu'))
model9.add(BatchNormalization())
model9.add(MaxPooling2D(2,2))
model9.add(Dropout(0.4))

model9.add(Flatten())

model9.add(Dense(32,activation='relu'))
model9.add(Dropout(0.4))
model9.add(Dense(1,activation='sigmoid'))
model9.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom9 = model9.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 353s 2s/step - loss: 0.7185 - acc: 0.5878 - val_loss: 0.9234 - val_acc: 0.5125
Epoch 2/10
200/200 [==============================] - 337s 2s/step - loss: 0.5658 - acc: 0.7202 - val_loss: 0.9609 - val_acc: 0.6922
Epoch 3/10
200/200 [==============================] - 351s 2s/step - loss: 0.3531 - acc: 0.8667 - val_loss: 0.2609 - val_acc: 0.9219
Epoch 4/10
200/200 [==============================] - 341s 2s/step - loss: 0.2487 - acc: 0.9169 - val_loss: 0.3267 - val_acc: 0.9250
Epoch 5/10
200/200 [==============================] - 349s 2s/step - loss: 0.2196 - acc: 0.9277 - val_loss: 0.3048 - val_acc: 0.9203
Epoch 6/10
200/200 [==============================] - 353s 2s/step - loss: 0.2036 - acc: 0.9354 - val_loss: 0.2642 - val_acc: 0.9359
Epoch 7/10
200/200 [==============================] - 336s 2s/step - loss: 0.1865 - acc: 0.9408 - val_loss: 0.1941 - val_acc: 0.9422
Epoch 8/10
200/200 [==============================] - 349s 2s/step - 

In [23]:
model9.save("Malaria_CNN_Trained9.h5")

In [24]:
# added another hidden convolutional layer with 32 filters
# model accuracies did not increase by a significant amount (used 70/30 training validation split)
# should stick with model 7 
model10 = Sequential()
model10.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model10.add(BatchNormalization())
model10.add(MaxPooling2D(2,2))
model10.add(Dropout(0.4))

model10.add(Convolution2D(64,(3,3),activation='relu'))
model10.add(BatchNormalization())
model10.add(MaxPooling2D(2,2))
model10.add(Dropout(0.4))

model10.add(Convolution2D(32,(3,3),activation='relu'))
model10.add(BatchNormalization())
model10.add(MaxPooling2D(2,2))
model10.add(Dropout(0.4))

model10.add(Convolution2D(32,(3,3),activation='relu'))
model10.add(BatchNormalization())
model10.add(MaxPooling2D(2,2))
model10.add(Dropout(0.4))

model10.add(Convolution2D(64,(3,3),activation='relu'))
model10.add(BatchNormalization())
model10.add(MaxPooling2D(2,2))
model10.add(Dropout(0.4))

model10.add(Flatten())

model10.add(Dense(32,activation='relu'))
model10.add(Dropout(0.4))
model10.add(Dense(1,activation='sigmoid'))
model10.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom10 = model10.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 366s 2s/step - loss: 0.8309 - acc: 0.5491 - val_loss: 0.8394 - val_acc: 0.6188
Epoch 2/10
200/200 [==============================] - 387s 2s/step - loss: 0.6527 - acc: 0.6306 - val_loss: 1.1778 - val_acc: 0.5156
Epoch 3/10
200/200 [==============================] - 411s 2s/step - loss: 0.5117 - acc: 0.7456 - val_loss: 1.0495 - val_acc: 0.6859
Epoch 4/10
200/200 [==============================] - 393s 2s/step - loss: 0.3464 - acc: 0.8731 - val_loss: 0.3363 - val_acc: 0.8812
Epoch 5/10
200/200 [==============================] - 341s 2s/step - loss: 0.2401 - acc: 0.9209 - val_loss: 0.2346 - val_acc: 0.9328
Epoch 6/10
200/200 [==============================] - 343s 2s/step - loss: 0.2163 - acc: 0.9367 - val_loss: 0.3236 - val_acc: 0.9156
Epoch 7/10
200/200 [==============================] - 339s 2s/step - loss: 0.1953 - acc: 0.9422 - val_loss: 0.2904 - val_acc: 0.9187
Epoch 8/10
200/200 [==============================] - 341s 2s/step - 

In [54]:
model10.save("Malaria_CNN_Trained10.h5")

In [50]:
# removed a hidden convolutional layer
# changed number of filters to 64, 32, 64, 32, 64
# model accuracies did not increase by a significant amount (used 70/30 training validation split)
# should stick with model 7 

model11 = Sequential()
model11.add(Convolution2D(64,(3,3),activation='relu',input_shape = (96,96,3)))
model11.add(BatchNormalization())
model11.add(MaxPooling2D(2,2))
model11.add(Dropout(0.4))

model11.add(Convolution2D(32,(3,3),activation='relu'))
model11.add(BatchNormalization())
model11.add(MaxPooling2D(2,2))
model11.add(Dropout(0.4))

model11.add(Convolution2D(64,(3,3),activation='relu'))
model11.add(BatchNormalization())
model11.add(MaxPooling2D(2,2))
model11.add(Dropout(0.4))

model11.add(Convolution2D(32,(3,3),activation='relu'))
model11.add(BatchNormalization())
model11.add(MaxPooling2D(2,2))
model11.add(Dropout(0.4))

model11.add(Flatten())

model11.add(Dense(64,activation='relu'))
model11.add(Dropout(0.4))
model11.add(Dense(1,activation='sigmoid'))
model11.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom11 = model11.fit_generator(train_generator, steps_per_epoch=200,
                              epochs = 10,validation_data=test_generator, validation_steps=20)

Epoch 1/10
200/200 [==============================] - 603s 3s/step - loss: 0.7026 - acc: 0.6314 - val_loss: 1.0437 - val_acc: 0.7031
Epoch 2/10
200/200 [==============================] - 579s 3s/step - loss: 0.4521 - acc: 0.7861 - val_loss: 0.2388 - val_acc: 0.9045
Epoch 3/10
200/200 [==============================] - 584s 3s/step - loss: 0.2770 - acc: 0.8952 - val_loss: 0.1983 - val_acc: 0.9313
Epoch 4/10
200/200 [==============================] - 589s 3s/step - loss: 0.2272 - acc: 0.9272 - val_loss: 0.2200 - val_acc: 0.9344
Epoch 5/10
200/200 [==============================] - 582s 3s/step - loss: 0.2135 - acc: 0.9311 - val_loss: 0.3023 - val_acc: 0.9156
Epoch 6/10
200/200 [==============================] - 576s 3s/step - loss: 0.1876 - acc: 0.9439 - val_loss: 0.2123 - val_acc: 0.9391
Epoch 7/10
200/200 [==============================] - 562s 3s/step - loss: 0.1846 - acc: 0.9445 - val_loss: 0.1650 - val_acc: 0.9469
Epoch 8/10
200/200 [==============================] - 560s 3s/step - 

In [55]:
model11.save("Malaria_CNN_Trained11.h5")

In [51]:
# increased steps per epoch from 200 to 500 (everything else is from model 11)
# increased validation steps from 20 to 50
# model accuracies did not increase by a significant amount (used 70/30 training validation split)
# should stick with model 7 

model12 = Sequential()
model12.add(Convolution2D(64,(3,3),activation='relu',input_shape = (96,96,3)))
model12.add(BatchNormalization())
model12.add(MaxPooling2D(2,2))
model12.add(Dropout(0.4))

model12.add(Convolution2D(32,(3,3),activation='relu'))
model12.add(BatchNormalization())
model12.add(MaxPooling2D(2,2))
model12.add(Dropout(0.4))

model12.add(Convolution2D(64,(3,3),activation='relu'))
model12.add(BatchNormalization())
model12.add(MaxPooling2D(2,2))
model12.add(Dropout(0.4))

model12.add(Convolution2D(32,(3,3),activation='relu'))
model12.add(BatchNormalization())
model12.add(MaxPooling2D(2,2))
model12.add(Dropout(0.4))

model12.add(Flatten())

model12.add(Dense(64,activation='relu'))
model12.add(Dropout(0.4))
model12.add(Dense(1,activation='sigmoid'))
model12.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom12 = model12.fit_generator(train_generator, steps_per_epoch=500,
                              epochs = 10,validation_data=test_generator, validation_steps=50)

Epoch 1/10
500/500 [==============================] - 1479s 3s/step - loss: 0.5683 - acc: 0.7123 - val_loss: 1.0512 - val_acc: 0.8113
Epoch 2/10
500/500 [==============================] - 1538s 3s/step - loss: 0.2407 - acc: 0.9192 - val_loss: 0.2354 - val_acc: 0.9303
Epoch 3/10
500/500 [==============================] - 1553s 3s/step - loss: 0.1910 - acc: 0.9396 - val_loss: 0.2146 - val_acc: 0.9481
Epoch 4/10
500/500 [==============================] - 1485s 3s/step - loss: 0.1794 - acc: 0.9458 - val_loss: 0.3489 - val_acc: 0.9131
Epoch 5/10
500/500 [==============================] - 1496s 3s/step - loss: 0.1651 - acc: 0.9511 - val_loss: 0.1605 - val_acc: 0.9450
Epoch 6/10
500/500 [==============================] - 1506s 3s/step - loss: 0.1571 - acc: 0.9504 - val_loss: 0.2065 - val_acc: 0.9444
Epoch 7/10
500/500 [==============================] - 1496s 3s/step - loss: 0.1596 - acc: 0.9499 - val_loss: 0.1725 - val_acc: 0.9442
Epoch 8/10
500/500 [==============================] - 1478s 3s

In [56]:
model12.save("Malaria_CNN_Trained12.h5")

In [52]:
# increased steps per epoch from 200 to 500 (everything else is from model 7)
# increased validation steps from 20 to 50
# model accuracies became slightly unstable
# should stick with model 7 

model13 = Sequential()
model13.add(Convolution2D(32,(3,3),activation='relu',input_shape = (96,96,3)))
model13.add(BatchNormalization())
model13.add(MaxPooling2D(2,2))
model13.add(Dropout(0.4))

model13.add(Convolution2D(64,(3,3),activation='relu'))
model13.add(BatchNormalization())
model13.add(MaxPooling2D(2,2))
model13.add(Dropout(0.4))

model13.add(Convolution2D(32,(3,3),activation='relu'))
model13.add(BatchNormalization())
model13.add(MaxPooling2D(2,2))
model13.add(Dropout(0.4))

model13.add(Flatten())

model13.add(Dense(64,activation='relu'))
model13.add(Dropout(0.4))
model13.add(Dense(1,activation='sigmoid'))
model13.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history_custom13 = model13.fit_generator(train_generator, steps_per_epoch=500,
                              epochs = 10,validation_data=test_generator, validation_steps=50)

Epoch 1/10
500/500 [==============================] - 981s 2s/step - loss: 0.6536 - acc: 0.6362 - val_loss: 0.5486 - val_acc: 0.7556
Epoch 2/10
500/500 [==============================] - 863s 2s/step - loss: 0.3462 - acc: 0.8508 - val_loss: 0.2418 - val_acc: 0.9202
Epoch 3/10
500/500 [==============================] - 888s 2s/step - loss: 0.1915 - acc: 0.9399 - val_loss: 0.7857 - val_acc: 0.5819
Epoch 4/10
500/500 [==============================] - 964s 2s/step - loss: 0.1717 - acc: 0.9459 - val_loss: 0.1751 - val_acc: 0.9381
Epoch 5/10
500/500 [==============================] - 874s 2s/step - loss: 0.1574 - acc: 0.9517 - val_loss: 0.3479 - val_acc: 0.8344
Epoch 6/10
500/500 [==============================] - 842s 2s/step - loss: 0.1524 - acc: 0.9534 - val_loss: 0.1562 - val_acc: 0.9525
Epoch 7/10
500/500 [==============================] - 921s 2s/step - loss: 0.1473 - acc: 0.9541 - val_loss: 0.2070 - val_acc: 0.9290
Epoch 8/10
500/500 [==============================] - 867s 2s/step - 

In [57]:
model13.save("Malaria_CNN_Trained13.h5")

#### Model 7 is the best of the ones trained. Some other models have slightly higher accuracies, but they had an extra hidden layer. The small increase in accuracy was not high enough to justify adding another hidden layer. In this case, it is better to use the slightly simpler model.